In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress
from scipy import stats
import scipy.stats as stats
%matplotlib inline

In [ ]:
#Bernt's code starts here

In [ ]:
# Importing necessary dataframes

birthrate_csv = "../project-one/Resources/children_per_woman_total_fertility.csv"
birthrate_df = pd.read_csv(birthrate_csv)
birthrate_df.head()

In [ ]:
co2_csv = "../project-one/Resources/co2_emissions_tonnes_per_person.csv"
co2_df = pd.read_csv(co2_csv)
co2_df.head()

In [ ]:
income_csv = "../project-one/Resources/income_per_person_gdppercapita_ppp_inflation_adjusted.csv"
income_df = pd.read_csv(income_csv)
income_df.head()

In [ ]:
population_csv = "../project-one/Resources/population_total.csv"
population_df = pd.read_csv(population_csv)
population_df.head()

In [ ]:
#Bernt's code ends here

In [ ]:
#Molly's code starts here

In [ ]:
# reshape the income dataframe to show every Country|Year with the income for that year (using group by looked nice but didn't cut it) 
df_meltInc =income_df.melt(id_vars=['country'], var_name='year', value_name='income')
df_meltInc = df_meltInc.dropna()
df_meltInc

In [ ]:
# take the reshaped income values and create a new data element that combines country and year
cols = ['country', 'year']
df_meltInc['combined'] = df_meltInc[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_meltInc

In [ ]:
# reshape the income dataframe to show every Country|Year with the income for that year (using group by looked nice but didn't cut it) 
df_meltCO2 =income_df.melt(id_vars=['country'], var_name='year', value_name='co2')
df_meltCO2 = df_meltCO2.dropna()
df_meltCO2

In [ ]:
# take the reshaped co2 values and create a new data element that combines country and year
cols = ['country', 'year']
df_meltCO2['combined'] = df_meltInc[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_meltCO2

In [ ]:
# merge the reshaped income and co2
merge_inc_co2_df = pd.merge(df_meltInc,df_meltCO2, on="combined", how="inner")
merge_inc_co2_df = merge_inc_co2_df.drop('country_x', axis=1)
merge_inc_co2_df = merge_inc_co2_df.drop('year_x', axis=1)
merge_inc_co2_df = merge_inc_co2_df.drop('country_y', axis=1)
merge_inc_co2_df = merge_inc_co2_df.drop('year_y', axis=1)
merge_inc_co2_df

In [ ]:
# get the log of the income so the span of the plot isn't so wide
merge_inc_co2_df['log income'] = np.log10(merge_inc_co2_df['income'])
merge_inc_co2_df.head()

In [ ]:
# plot the log of the income and the co2 to see if there was correlation
x_axis = merge_inc_co2_df["log income"]
y_axis = merge_inc_co2_df["co2"]
plt.xlabel("Log Income")
plt.ylabel("CO2 Emissions")
plt.title("Impact of Income on CO2 Emissions 1800 to 2014")
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.scatter(x_axis, y_axis, alpha=0.5, s=10)
plt.savefig(os.path.join("Images", "ImpactIncomeOnCO2.png"))
plt.show()

In [ ]:
print(merge_inc_co2_df['income'].mean())
print(merge_inc_co2_df['income'].median())
print(merge_inc_co2_df['income'].mode())

In [ ]:
print(merge_inc_co2_df['co2'].mean())
print(merge_inc_co2_df['co2'].median())
print(merge_inc_co2_df['co2'].mode())

In [ ]:
# Determine if the co2 emissions is normally distributed
plt.hist(merge_inc_co2_df['co2'])
plt.xlabel('co2')
plt.ylabel('Counts')
plt.show()
print(stats.normaltest(merge_inc_co2_df["co2"].sample(100)))

In [ ]:
# reshape the population dataframe to show every Country|Year with the population for that year (using group by looked nice but didn't cut it) 
df_meltPop = population_df.melt(id_vars=['country'], var_name='year', value_name='population')
df_meltPop = df_meltPop.dropna()
df_meltPop

In [ ]:
# take the reshaped population values and create a new data element that combines country and year
cols = ['country', 'year']
df_meltPop['combined'] = df_meltPop[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_meltPop

In [ ]:
# reshape the birthrate dataframe to show every Country|Year with the birthrate for that year (using group 
# by looked nice but didn't cut it) 
df_meltBirth = birthrate_df.melt(id_vars=['country'], var_name='year', value_name='birth rate')
df_meltBirth = df_meltBirth.dropna()
df_meltBirth

In [ ]:
# take the reshaped birth rate values and create a new data element that combines country and year
cols = ['country', 'year']
df_meltBirth['combined'] = df_meltBirth[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_meltBirth

In [ ]:
# merge the reshaped population and birth rate
merge_pop_birth_df = pd.merge(df_meltPop,df_meltBirth, on="combined", how="inner")
merge_pop_birth_df = merge_pop_birth_df.drop('country_x', axis=1)
merge_pop_birth_df = merge_pop_birth_df.drop('year_x', axis=1)
merge_pop_birth_df = merge_pop_birth_df.drop('country_y', axis=1)
merge_pop_birth_df = merge_pop_birth_df.drop('year_y', axis=1)
merge_pop_birth_df

In [ ]:
# merge the income/co2 dataframe to the population/birth rate dataframe
merge_all_df = pd.merge(merge_pop_birth_df,merge_inc_co2_df, on="combined", how="inner")
merge_all_df

In [ ]:
# plot the birth rate and the population to see if there was correlation
x_axis = merge_all_df["birth rate"]
y_axis = merge_all_df["population"]
plt.xlabel("Birth Rate")
plt.ylabel("Population")
plt.title("Impact of Birth Rate on Population 1800 to 2014")
plt.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
plt.scatter(x_axis, y_axis, alpha=0.5, s=10)
plt.savefig(os.path.join("Images","ImpactBirthOnPop.png"))
plt.show()

In [ ]:
# merge the reshaped birth rate and co2
merge_birth_co2_df = pd.merge(df_meltBirth, df_meltCO2, on="combined", how="inner")
merge_birth_co2_df = merge_birth_co2_df.drop('country_x', axis=1)
merge_birth_co2_df = merge_birth_co2_df.drop('year_x', axis=1)
merge_birth_co2_df = merge_birth_co2_df.drop('country_y', axis=1)
merge_birth_co2_df = merge_birth_co2_df.drop('year_y', axis=1)
merge_birth_co2_df

In [ ]:
# plot the CO2 and birth rate to see if there was correlation
x_values = merge_birth_co2_df["co2"]
y_values = merge_birth_co2_df["birth rate"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.savefig(os.path.join("Images","ImpactBirthOnPop.png"))
plt.title("CO2 Emissions v Birthrate (1800 - 2014)")

In [ ]:
# perform a linear regression on co2 v birthrate 
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel('CO2 Emissions')
plt.ylabel('Birthrate')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (1800-2014)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","allYears_co2_v_birthrate.png"))

# show plot
plt.show()

In [ ]:
#Molly's code ends here

In [ ]:
#Kaan's code starts here

In [ ]:
#Kaan's code ends here

In [ ]:
#Vallie's code starts here

In [ ]:
# print out the birthrate df
birthrate_df.head()

In [ ]:
# print out the co2 dataframe
co2_df.head()

In [ ]:
# merge birthrate and co2 dataframes on country
co2_br_merge = pd.merge(co2_df, birthrate_df, on="country")
co2_br_merge.head()

In [ ]:
# rename columns
co2_br_rename = co2_br_merge.rename(columns={"country": "Country",
                                             "1974_x": "1974 CO2 Emissions",
                                             "1984_x": "1984 CO2 Emissions",
                                             "1994_x": "1994 CO2 Emissions",
                                             "2004_x": "2004 CO2 Emissions",
                                             "2014_x": "2014 CO2 Emissions",
                                             "1974_y": "Birthrate_1974",
                                             "1984_y": "Birthrate_1984",
                                             "1994_y": "Birthrate_1994",
                                             "2004_y": "Birthrate_2004",
                                             "2014_y": "Birthrate_2014"})

# make df with only the columns we want to keep
co2_br_df = co2_br_rename[["Country", "1974 CO2 Emissions", "1984 CO2 Emissions", "1994 CO2 Emissions", "2004 CO2 Emissions", "2014 CO2 Emissions",
                           "Birthrate_1974", "Birthrate_1984", "Birthrate_1994", "Birthrate_2004", "Birthrate_2014"]]
co2_br_df.head()

In [ ]:
# create 1974 dataframe, co2 descending order
df_1974 = co2_br_df[["Country", "1974 CO2 Emissions", "Birthrate_1974"]].sort_values("1974 CO2 Emissions", ascending=False).reset_index()
df_1974 = df_1974[["Country", "1974 CO2 Emissions", "Birthrate_1974"]]

# drop rows with nan values
df_1974 = df_1974.dropna()
df_1974

In [ ]:
# run linear regression on co2 v birthrate (year 1974)

# first create scatter plot
x_values = df_1974["1974 CO2 Emissions"]
y_values = df_1974["Birthrate_1974"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.title("CO2 Emissions v Birthrate (1974)")

In [ ]:
# perform a linear regression on co2 v birthrate (year 1974)
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate, 1974)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel('CO2 Emissions')
plt.ylabel('Birthrate')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (1974)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","1974_co2_v_birthrate.png"))

# show plot
plt.show()

In [ ]:
# create 1984 dataframe, co2 descending order
df_1984 = co2_br_df[["Country", "1984 CO2 Emissions", "Birthrate_1984"]].sort_values("1984 CO2 Emissions", ascending=False).reset_index()
df_1984 = df_1984[["Country", "1984 CO2 Emissions", "Birthrate_1984"]]

# drop rows with nan values
df_1984 = df_1984.dropna()
df_1984.head()

In [ ]:
# run linear regression on co2 v birthrate (year 1984)

# first create scatter plot
x_values = df_1984["1984 CO2 Emissions"]
y_values = df_1984["Birthrate_1984"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.title("CO2 Emissions v Birthrate (1984)")

In [ ]:
# perform a linear regression on co2 v birthrate (year 1984)
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate, 1984)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel('CO2 Emissions')
plt.ylabel('Birthrate')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (1984)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","1984_co2_v_birthrate.png"))

# show plot
plt.show()

In [ ]:
# create 1994 dataframe, co2 descending order
df_1994 = co2_br_df[["Country", "1994 CO2 Emissions", "Birthrate_1994"]].sort_values("1994 CO2 Emissions", ascending=False).reset_index()
df_1994 = df_1994[["Country", "1994 CO2 Emissions", "Birthrate_1994"]]

# drop rows with nan values
df_1994 = df_1994.dropna()
df_1994.head()

In [ ]:
# run linear regression on co2 v birthrate (year 1994)

# first create scatter plot
x_values = df_1994["1994 CO2 Emissions"]
y_values = df_1994["Birthrate_1994"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.title("CO2 Emissions v Birthrate (1994)")

In [ ]:
# perform a linear regression on co2 v birthrate (year 1994)
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate, 1994)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel("CO2 Emissions")
plt.ylabel("Birthrate")
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (1994)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","1994_co2_v_birthrate.png"))

# show plot
plt.show()

In [ ]:
# create 2004 dataframe, co2 descending order
df_2004 = co2_br_df[["Country", "2004 CO2 Emissions", "Birthrate_2004"]].sort_values("2004 CO2 Emissions", ascending=False).reset_index()
df_2004 = df_2004[["Country", "2004 CO2 Emissions", "Birthrate_2004"]]

# drop rows with nan values
df_2004 = df_2004.dropna()
df_2004.head()

In [ ]:
# run linear regression on co2 v birthrate (year 2004)

# first create scatter plot
x_values = df_2004["2004 CO2 Emissions"]
y_values = df_2004["Birthrate_2004"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.title("CO2 Emissions v Birthrate (2004)")

In [ ]:
# perform a linear regression on co2 v birthrate (year 2004)
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate, 2004)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel('CO2 Emissions')
plt.ylabel('Birthrate')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (2004)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","2004_co2_v_birthrate.png"))

# show plot
plt.show()

In [ ]:
# create 2014 dataframe, co2 descending order
df_2014 = co2_br_df[["Country", "2014 CO2 Emissions", "Birthrate_2014"]].sort_values("2014 CO2 Emissions", ascending=False).reset_index()
df_2014 = df_2014[["Country", "2014 CO2 Emissions", "Birthrate_2014"]]

# drop rows with nan values
df_2014 = df_2014.dropna()
df_2014.head()

In [ ]:
# run linear regression on co2 v birthrate (year 2014)

# first create scatter plot
x_values = df_2014["2014 CO2 Emissions"]
y_values = df_2014["Birthrate_2014"]
plt.scatter(x_values, y_values, edgecolors="black")
plt.xlabel("CO2 Emissions (tons/person)")
plt.ylabel("Birthrate (child/woman)")
plt.title("CO2 Emissions v Birthrate (2014)")

In [ ]:
# perform a linear regression on co2 v birthrate (year 2014)
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# get regression values
regress_values = x_values * slope + intercept

# create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
print(line_eq)

In [ ]:
# create plot (co2 v birthrate, 2014)
plt.scatter(x_values,y_values,edgecolors="black")
plt.plot(x_values,regress_values,"r-")

# label plot and annotate the line equation
plt.xlabel('CO2 Emissions')
plt.ylabel('Birthrate')
plt.annotate(line_eq,(20,5),fontsize=15,color="red")
plt.title("CO2 Emissions v Birthrate (2014)")

# Print r square value
print(f"The r-squared is: {rvalue}")

# save plot to images folder
plt.savefig(os.path.join("Images","2014_co2_v_birthrate.png"))

# show plot
plt.show()

**ANALYSIS:**   Based off of linear regression alone, there doesn't appear to be a statistically significant correlation between co2 and birthrate, over any of the years Kaan and I looked at. Could it be of significance that 4 of the 5 years I tested had an r-value that fell within a .058 window of each other? 1974 was the farthest away from the rest, with an r-value of -.28, classified as a weak correlation (the rest fell under a 'moderate' correlation).

In [ ]:
# running anova and independent t-tests (1974 co2 v birthrate)

# create bins to place the carbon emissions
bins = [0, 22.72, 45.46, 69]

# create labels for the bins
group_labels = ["Low-Level CO2", "Mid-Level CO2", "High-Level CO2"]

# slice the data and place it into bins
pd.cut(df_1974["1974 CO2 Emissions"], bins, labels=group_labels)

# place the data series into a new column inside of the dataframe
df_1974["CO2 Group"] = pd.cut(df_1974["1974 CO2 Emissions"], bins, labels=group_labels)
df_1974.head()

In [ ]:
# anova (co2 v birthrate, 1974)

# create a boxplot to compare means
df_1974.boxplot("Birthrate_1974", by="CO2 Group", figsize=(20, 10))

In [ ]:
# extract individual groups
group1 = df_1974[df_1974["CO2 Group"] == "Low-Level CO2"]["Birthrate_1974"]
group2 = df_1974[df_1974["CO2 Group"] == "Mid-Level CO2"]["Birthrate_1974"]
group3 = df_1974[df_1974["CO2 Group"] == "High-Level CO2"]["Birthrate_1974"]

# perform the anova
stats.f_oneway(group1, group2, group3)

In [ ]:
# independent t-test

# calculate the 'population' mean for birthrate in lowest co2 group (year 1974)
population1 = df_1974[df_1974["CO2 Group"] == "Low-Level CO2"]
population1.Birthrate_1974.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 1974)
population2 = df_1974[df_1974["CO2 Group"] == "Mid-Level CO2"]
population2.Birthrate_1974.mean()

In [ ]:
# calculate the 'population' mean for birthrate in highest co2 group (year 1974)
population3 = df_1974[df_1974["CO2 Group"] == "High-Level CO2"]
population3.Birthrate_1974.mean()

In [ ]:
# calculate independent t-test, low v mid (1974)
stats.ttest_ind(population1.Birthrate_1974, population2.Birthrate_1974, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, low v high (1974)
stats.ttest_ind(population1.Birthrate_1974, population3.Birthrate_1974, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, mid v high (1974)
stats.ttest_ind(population2.Birthrate_1974, population3.Birthrate_1974, equal_var=False)

**ANALYSIS:** Looking at 1974, we fail to reject the null hypothesis that all three co2 level groups have the same mean birthrate. But when comparing the groups to one another, we found that there could be statistically significant data in the low v high comparison.

In [ ]:
# running anova and independent t-tests (1984 co2 v birthrate)

# create bins to place the carbon emissions
bins = [0, 12.06, 24.08, 36.2]

# create labels for the bins
group_labels = ["Low-Level CO2", "Mid-Level CO2", "High-Level CO2"]

# slice the data and place it into bins
pd.cut(df_1984["1984 CO2 Emissions"], bins, labels=group_labels)

# place the data series into a new column inside of the DataFrame
df_1984["CO2 Group"] = pd.cut(df_1984["1984 CO2 Emissions"], bins, labels=group_labels)
df_1984.head()

In [ ]:
# anova (co2 v birthrate, 1984)

# create a boxplot to compare means
df_1984.boxplot("Birthrate_1984", by="CO2 Group", figsize=(20, 10))

In [ ]:
# extract individual groups
group1 = df_1984[df_1984["CO2 Group"] == "Low-Level CO2"]["Birthrate_1984"]
group2 = df_1984[df_1984["CO2 Group"] == "Mid-Level CO2"]["Birthrate_1984"]
group3 = df_1984[df_1984["CO2 Group"] == "High-Level CO2"]["Birthrate_1984"]

# perform the anova
stats.f_oneway(group1, group2, group3)

In [ ]:
# independent t-test

# calculate the 'population' mean for birthrate in lowest co2 group (year 1984)
population1 = df_1984[df_1984["CO2 Group"] == "Low-Level CO2"]
population1.Birthrate_1984.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 1984)
population2 = df_1984[df_1984["CO2 Group"] == "Mid-Level CO2"]
population2.Birthrate_1984.mean()

In [ ]:
# calculate the 'population' mean for birthrate in highest co2 group (year 1984)
population3 = df_1984[df_1984["CO2 Group"] == "High-Level CO2"]
population3.Birthrate_1984.mean()

In [ ]:
#calculate independent (two sample) t-test, low v mid (1984)
stats.ttest_ind(population1.Birthrate_1984, population2.Birthrate_1984, equal_var=False)

In [ ]:
#calculate independent (two sample) t-test, low v high (1984)
stats.ttest_ind(population1.Birthrate_1984, population3.Birthrate_1984, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, mid v high (1984)
stats.ttest_ind(population2.Birthrate_1984, population3.Birthrate_1984, equal_var=False)

**ANALYSIS:** The 1984 anova test has a statistically significant p-value of 0.0002, meaning we reject the null hypothesis. When comparing the three co2 groups against one another, low level versus mid level is the only comparison where we also rejected the null hypothesis. Not being well-versed in statistics, could it be worthwhile to further explore why the anova test rejected the null hypothesis but the t-test only rejected the null hypothesis on 1 out of the 3 comparisons?

In [ ]:
# running anova and independent t-tests (1994 co2 v birthrate)

# create bins to place the carbon emissions
# changed bins to pull out singleton high level co2
bins = [0, 10.47, 20.94, 31.5]

# create labels for the bins
group_labels = ["Low-Level CO2", "Mid-Level CO2", "High-Level CO2"]

# slice the data and place it into bins
pd.cut(df_1994["1994 CO2 Emissions"], bins, labels=group_labels)

#place the data series into a new column inside of the DataFrame
df_1994["CO2 Group"] = pd.cut(df_1994["1994 CO2 Emissions"], bins, labels=group_labels)
df_1994.head()

In [ ]:
# anova (co2 v birthrate, 1994)

# create a boxplot to compare means
df_1994.boxplot("Birthrate_1994", by="CO2 Group", figsize=(20, 10))

In [ ]:
# extract individual groups
group1 = df_1994[df_1994["CO2 Group"] == "Low-Level CO2"]["Birthrate_1994"]
group2 = df_1994[df_1994["CO2 Group"] == "Mid-Level CO2"]["Birthrate_1994"]
group3 = df_1994[df_1994["CO2 Group"] == "High-Level CO2"]["Birthrate_1994"]

# perform the anova
stats.f_oneway(group1, group2, group3)

In [ ]:
# independent t-test

# calculate the 'population' mean for birthrate in lowest co2 group (year 1994)
population1 = df_1994[df_1994["CO2 Group"] == "Low-Level CO2"]
population1.Birthrate_1994.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 1994)
population2 = df_1994[df_1994["CO2 Group"] == "Mid-Level CO2"]
population2.Birthrate_1994.mean()

In [ ]:
# calculate the 'population' mean for birthrate in highest co2 group (year 1994)
population3 = df_1994[df_1994["CO2 Group"] == "High-Level CO2"]
population3.Birthrate_1994.mean()

In [ ]:
# calculate independent (two sample) t-test, low v mid (1994)
stats.ttest_ind(population1.Birthrate_1994, population2.Birthrate_1994, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, low v high (1994)
stats.ttest_ind(population1.Birthrate_1994, population3.Birthrate_1994, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, mid v high (1994)
stats.ttest_ind(population2.Birthrate_1994, population3.Birthrate_1994, equal_var=False)

**ANALYSIS:** Similar to 1984, we reject the null hypothesis that the three co2 groups have the same birthrate means. This is a statistically significant result. And running the independent t-test, we found one p-value under .05, looking at the low versus mid co2 groups.

*Important to note:* We pulled out a singleton high-level co2 value from the 1994 data. Is it worth exploring further, maybe breaking the data into smaller subsets, or running different statistical tests?

In [ ]:
# running anova and independent t-tests (2004 co2 v birthrate)

# create bins to place the carbon emissions
# change bins in order to pull out singleton high level co2
bins = [0, 9.43, 18.87, 28.9]

# create labels for the bins
group_labels = ["Low-Level CO2", "Mid-Level CO2", "High-Level CO2"]

# slice the data and place it into bins
pd.cut(df_2004["2004 CO2 Emissions"], bins, labels=group_labels)

# place the data series into a new column inside of the DataFrame
df_2004["CO2 Group"] = pd.cut(df_2004["2004 CO2 Emissions"], bins, labels=group_labels)
df_2004.head()

In [ ]:
# anova (co2 v birthrate, 2004)

# create a boxplot to compare means
df_2004.boxplot("Birthrate_2004", by="CO2 Group", figsize=(20, 10))

In [ ]:
# extract individual groups
group1 = df_2004[df_2004["CO2 Group"] == "Low-Level CO2"]["Birthrate_2004"]
group2 = df_2004[df_2004["CO2 Group"] == "Mid-Level CO2"]["Birthrate_2004"]
group3 = df_2004[df_2004["CO2 Group"] == "High-Level CO2"]["Birthrate_2004"]

# perform the anova
stats.f_oneway(group1, group2, group3)

In [ ]:
# independent t-test

# calculate the 'population' mean for birthrate in lowest co2 group (year 2004)
population1 = df_2004[df_2004["CO2 Group"] == "Low-Level CO2"]
population1.Birthrate_2004.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 2004)
population2 = df_2004[df_2004["CO2 Group"] == "Mid-Level CO2"]
population2.Birthrate_2004.mean()

In [ ]:
# calculate the 'population' mean for birthrate in highest co2 group (year 2004)
population3 = df_2004[df_2004["CO2 Group"] == "High-Level CO2"]
population3.Birthrate_2004.mean()

In [ ]:
# calculate independent (two sample) t-test, low v mid (2004)
stats.ttest_ind(population1.Birthrate_2004, population2.Birthrate_2004, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, low v high (2004)
stats.ttest_ind(population1.Birthrate_2004, population3.Birthrate_2004, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, mid v high (2004)
stats.ttest_ind(population2.Birthrate_2004, population3.Birthrate_2004, equal_var=False)

**ANALYSIS:** In line with 1984 and 1994, the anova produced a rejection of the null hypothesis. And then testing the groups against one another, we also reject the null hypothesis as it pertains to the low v mid co2 group and low v high co2 group.

*Important to note:* Similar to 1994, we pulled out a singleton high-level co2 datapoint.

In [ ]:
# running anova and independent t-tests (2014 co2 v birthrate)

# create bins to place the carbon emissions
bins = [0, 15.13, 30.26, 45.5]

# create labels for the bins
group_labels = ["Low-Level CO2", "Mid-Level CO2", "High-Level CO2"]

# slice the data and place it into bins
pd.cut(df_2014["2014 CO2 Emissions"], bins, labels=group_labels)

# place the data series into a new column inside of the DataFrame
df_2014["CO2 Group"] = pd.cut(df_2014["2014 CO2 Emissions"], bins, labels=group_labels)
df_2014.head()

In [ ]:
# anova (co2 v birthrate, 2014)

# create a boxplot to compare means
df_2014.boxplot("Birthrate_2014", by="CO2 Group", figsize=(20, 10))

In [ ]:
# extract individual groups
group1 = df_2014[df_2014["CO2 Group"] == "Low-Level CO2"]["Birthrate_2014"]
group2 = df_2014[df_2014["CO2 Group"] == "Mid-Level CO2"]["Birthrate_2014"]
group3 = df_2014[df_2014["CO2 Group"] == "High-Level CO2"]["Birthrate_2014"]

# perform the anova
stats.f_oneway(group1, group2, group3)

In [ ]:
# independent t-test

# calculate the 'population' mean for birthrate in lowest co2 group (year 2014)
population1 = df_2014[df_2014["CO2 Group"] == "Low-Level CO2"]
population1.Birthrate_2014.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 2014)
population2 = df_2014[df_2014["CO2 Group"] == "Mid-Level CO2"]
population2.Birthrate_2014.mean()

In [ ]:
# calculate the 'population' mean for birthrate in mid co2 group (year 2014)
population3 = df_2014[df_2014["CO2 Group"] == "High-Level CO2"]
population3.Birthrate_2014.mean()

In [ ]:
# calculate independent (two sample) t-test, low v mid (2014)
stats.ttest_ind(population1.Birthrate_2014, population2.Birthrate_2014, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, low v high (2014)
stats.ttest_ind(population1.Birthrate_2014, population3.Birthrate_2014, equal_var=False)

In [ ]:
# calculate independent (two sample) t-test, mid v high (2014)
stats.ttest_ind(population2.Birthrate_2014, population3.Birthrate_2014, equal_var=False)

**ANALYSIS:**  The anova test performed on the 2014 data set resulted in a failure to reject the null hypothesis. While this isn't statistically significant, we did find statistical significance when performing independent t-tests on both low v mid co2 groups and low v high co2 groups.

In [ ]:
#Vallie's code ends here